In [1]:
import numpy as np
import pandas as pd
import random
import time
import scipy as sp
from scipy.sparse.linalg import cg

In [2]:
from data_datasets import higgs, susy, cod_rna
N = 3000
data = "susy"
# data = "higgs"
# data = "cod_rna"
####################

if data == "higgs":
    X_train, X_test, y_train, y_test = higgs(N)
elif data == "susy":
    X_train, X_test, y_train, y_test = susy(N)
elif data == "cod_rna":
    X_train, X_test, y_train, y_test = cod_rna(N)

print("\nDataset:", data)
print("--------\nShape train data:", X_train.shape)
print("Shape test data:", X_test.shape)


Dataset: susy
--------
Shape train data: (3000, 18)
Shape test data: (3000, 18)


In [ ]:

import numpy as np
from scipy.sparse.linalg import cg
import time


def conjugate_grad(A, b, x=None):
    """
    Description
    -----------
    Solve a linear equation Ax = b with conjugate gradient method.
    Parameters
    ----------
    A: 2d numpy.array of positive semi-definite (symmetric) matrix
    b: 1d numpy.array
    x: 1d numpy.array of initial point
    Returns
    -------
    1d numpy.array x such that Ax = b
    """
    n = len(b)
    if not x:
        x = np.ones(n)
    r = np.dot(A, x) - b
    p = - r
    r_k_norm = np.dot(r, r)
    for i in range(2*n):
        Ap = np.dot(A, p)
        alpha = r_k_norm / np.dot(p, Ap)
        x += alpha * p
        r += alpha * Ap
        r_kplus1_norm = np.dot(r, r)
        beta = r_kplus1_norm / r_k_norm
        r_k_norm = r_kplus1_norm
        if r_kplus1_norm < 1e-5:
            print ('Itr:', i)
            break
        p = beta * p - r
    return x

# x_ph = tf.placeholder('float32', [None, None])
# r = tf.matmul(A, x_ph) - b

In [3]:
from sklearn.metrics.pairwise import rbf_kernel
from low_rank_methods import nystrom, rpcholesky, rff
from utils import WB_Identity


mu = .02
rank = 5

N, d = X_train.shape
I = np.eye(N)

K = rbf_kernel(X_train, gamma=.1)
U = nystrom(K, rank)[0]
D = nystrom(K, rank)[1]


In [4]:
K_nys = U@D@U.T

In [6]:
P = K_nys+mu*I

In [7]:
A = K+mu*I

In [ ]:
eig = np.linalg.eigvals(K_nys)

In [ ]:
P_nys= U@(D+mu*np.eye(rank))@U.T/(eig[rank]+mu)+I-U@U.T


In [ ]:
residuals = []
b = y_train


def callback(x): return residuals.append(
    np.linalg.norm(A @ x - b) / np.linalg.norm(b))


alpha, info = cg(A, b, M=P, callback=callback, tol=1e-3)

In [ ]:
D.shape, U.shape

In [8]:
from iterative_methods import CG
P_inv = WB_Identity(mu*I, U, U.T, D )
sol, residuals = CG(A=A, b=y_train, P_inv=P_inv)

0 [ 1. -1. -1. ... -1. -1.  1.]
1 [-31.36078117 230.57362053 -69.49356193 ...  64.52228913 141.8784516
 118.43410787]
2 [-30.9412628  228.38983872 -68.64376831 ...  63.92897175 140.4916372
 117.46024049]
3 [-30.94505148 228.40962382 -68.65144578 ...  63.93434983 140.50419757
 117.46907794]
4 [-30.94501715 228.40944455 -68.65137622 ...  63.93430111 140.50408377
 117.46899787]
5 [-30.94501747 228.40944618 -68.65137685 ...  63.93430155 140.5040848
 117.4689986 ]
6 [-30.94501746 228.40944616 -68.65137684 ...  63.93430154 140.50408479
 117.46899859]
7 [-30.94501746 228.40944616 -68.65137684 ...  63.93430154 140.50408479
 117.46899859]
8 [-30.94501746 228.40944616 -68.65137684 ...  63.93430154 140.50408479
 117.46899859]
9 [-30.94501746 228.40944616 -68.65137684 ...  63.93430154 140.50408479
 117.46899859]
10 [-30.94501746 228.40944616 -68.65137684 ...  63.93430154 140.50408479
 117.46899859]
11 [-30.94501746 228.40944616 -68.65137684 ...  63.93430154 140.50408479
 117.46899859]
12 [-30.9450

In [9]:
print(residuals[-1], len(residuals))
residuals

6574.967902815578 76


array([  54.77225575, 6635.57673271, 6574.41865224, 6574.97287929,
       6574.96785773, 6574.96790322, 6574.96790281, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790282,
       6574.96790282, 6574.96790282, 6574.96790282, 6574.96790